First use requests and BeautifulSoup to read the html of the Toronto postal codes into variable soup

In [1]:
import requests
from bs4 import BeautifulSoup
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

Use BeautifulSoup to read the html of the postalcode table into variable table. 
Then read the body of the table into variable table_body. 
Next, read the table rows into variable table_rows.
Then iterate through the table rows to read the text of the table elements (td) into the variable records. 
Display the contents of variable records

In [2]:
table=soup.find('table', class_='wikitable sortable')
table_body=table.find('tbody')
table_rows=table_body.find_all('tr')

del table_rows[0]         # delete empty row coming from header

records = []
for row in table_rows:
    cells = row.find_all('td')
    cells=[cell.text.strip() for cell in cells]   # find text inside td
    records.append(cells)

records

[['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M5A', 'Downtown Toronto', 'Regent Park'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', "Queen's Park", 'Not assigned'],
 ['M8A', 'Not assigned', 'Not assigned'],
 ['M9A', 'Etobicoke', 'Islington Avenue'],
 ['M1B', 'Scarborough', 'Rouge'],
 ['M1B', 'Scarborough', 'Malvern'],
 ['M2B', 'Not assigned', 'Not assigned'],
 ['M3B', 'North York', 'Don Mills North'],
 ['M4B', 'East York', 'Woodbine Gardens'],
 ['M4B', 'East York', 'Parkview Hill'],
 ['M5B', 'Downtown Toronto', 'Ryerson'],
 ['M5B', 'Downtown Toronto', 'Garden District'],
 ['M6B', 'North York', 'Glencairn'],
 ['M7B', 'Not assigned', 'Not assigned'],
 ['M8B', 'Not assigned', 'Not assigned'],
 ['M9B', 'Etobicoke', 'Cloverdale'],
 ['M9B', 'Etobicoke', 'Islington'],
 ['M9B', 

Set up a pandas dataframe variable df with column headers 'PostalCode', 'Borough', and 'Neighborhood'. Fill df with the contents of records. Where the value of 'Neighborhood' is 'Not assigned' then change that value to the value of 'Borough'. Group by 'PostalCode' and 'Borough' and combine 'Neighborhood' values separated by commas. Display the dataframe df.

In [3]:
import pandas as pd
headers = ['PostalCode','Borough','Neighborhood']
df = pd.DataFrame(records, columns=headers)
df = df[df['Borough'] != 'Not assigned']

# where neighborhood == 'Not assigned' make Neighborhood = Borough
cols = df.columns[df.columns.str.contains('Neighborhood')]
df[cols] = df[cols].mask(df[cols].apply(lambda x: x.str.contains('Not assigned')), df['Borough'], axis=0)

df = df.groupby(['PostalCode','Borough'], sort=False).agg(lambda x: ', '.join(x)).reset_index()
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


Print the number of rows of the dataframe

In [4]:
df.shape

(103, 3)